# PyCitySchool
*Created by Dale Currigan*

Scroll down or use the navigation links below:  
  
[District Summary](#district_summary)  
[School Summary](#school_summary)  
[Top Performing Schools](#top_performing_schools)  
[Bottoming Performing Schools](#bottom_performing_schools)  
[Average Math Score by Grade](#average_math_grade)  
[Average Reading Score by Grade](#average_reading_grade)  
[Scores by School Spending](#school_spending)  
[Scores by School Size](#school_size)  
[Scores by School Type](#school_type)  
[School Summary - Alternative Solution](#school_summary_alt) 

In [1]:
# Import libraries and setup
import pandas as pd
pd.set_option('mode.chained_assignment', None)

# Files to Load 
school_data_to_load = "Resources/schools_complete.csv"

student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into DataFrames
# Sort school df into alphabetical order
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single df (combined_df)  
combined_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])                               


In [2]:
combined_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


<a id='district_summary'></a>
## District Summary
A high level snapshot of the district's key metrics

In [3]:
# calculate total school number from index of school_data
total_schools = len(school_data)

# calculate total student number from index of student_data
total_students = len(student_data.index)

# calculate total budget as sum of individual school budgets
total_budget = school_data['budget'].sum()

# calculate average math score
average_math = combined_df['math_score'].mean()

# calculate average reading score
average_reading = combined_df['reading_score'].mean()

# caculate percent passing math by creating a df containing only those with scores >= 70 for math 
passing_math = combined_df[combined_df['math_score'] >= 70]
percent_passed_math = len(passing_math) / len(combined_df) * 100

# caculate percent passing reading by creating a df containing only those with scores >= 70 for reading 
passing_reading = combined_df[combined_df['reading_score'] >= 70]
percent_passed_reading = len(passing_reading) / len(combined_df) * 100

# calculate percent passing both
# start with the passing_math df and narrow down to only those with scores >= 70 for reading (i.e. passing both)
passing_both = passing_math[passing_math['reading_score'] >= 70]
percent_passing_both = len(passing_both) / len(combined_df) * 100

# merge results into a dataframe, format, and display
district_df = pd.DataFrame([{"Total Schools": total_schools, "Total Students": total_students, "Total Budget": total_budget,
                         "Average Math Score":average_math, "Average Reading Score": average_reading, "% Passing Math": percent_passed_math,
                         "% Passing Reading": percent_passed_reading, "% Overall Passing":percent_passing_both}])

district_df['Total Budget'] = district_df['Total Budget'].map("${:,.2f}".format)
district_df.iloc[:,3:8] = round(district_df.iloc[:,3:8],3)
district_df.iloc[:,5:8] = district_df.iloc[:,3:8].applymap("{}%".format)
district_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985,81.878,74.981%,85.805%,65.172%


<a id='school_summary'></a>
## School Summary
An overview of each schools key metrics

In [4]:
# SCHOOL NAME, TYPE, BUDGET, SIZE, AVERAGE MATH & READING SCORES
# can all be retrieved with a groupby object. This is tored in our primary dataframe (df)
schools_grouped = combined_df.groupby(['school_name', 'type','size', 'budget'])
df = pd.DataFrame(schools_grouped[['math_score','reading_score']].mean()).reset_index()


# BUDGET PER STUDENT - groupby school, budget and size, then divide budget by student number 
budget_data = combined_df.groupby(['school_name', 'budget', 'size']).size().reset_index(name=("count"))
# data inserted at column index 4 (next to total budget)
df.insert(4,'Budget Per Student',(budget_data['budget']/budget_data['size']))


# PERCENT PASSING EACH SUBJECT
#  create a df with the number of students in each school 
all_students = combined_df.groupby(['school_name']).size().reset_index(name=("count"))
#  then create df's with the counts for students passing each subject, at each school
#  code compoonents:  -- Filtered df of passing students--    -- grouped by school --   -- and then counted --  
passing_math = combined_df[combined_df['math_score'] >=70 ].groupby(['school_name']).size().reset_index(name=("count"))
passing_reading = combined_df[combined_df['reading_score'] >=70 ].groupby(['school_name']).size().reset_index(name=("count"))
passing_both = combined_df[((combined_df['math_score'] >=70) & (combined_df['reading_score']>=70))]
passing_both = passing_both.groupby(['school_name']).size().reset_index(name=("count"))


# finally, calculate percentage and append to df 
df['% Passing Math'] = passing_math['count'] / all_students['count']*100
df['% Passing Reading'] = passing_reading['count'] / all_students['count']*100
df['% Overall Passing'] = passing_both['count'] / all_students['count']*100

# UPDATE COLUMN HEADERS AND FORMATTING 
df.rename(columns={"school_name":"School", "type":"Type", "budget":"Total Budget", "size":"Total Students", 
                   "math_score":"Average Math Score", "reading_score":"Average Reading Score"}, inplace=True)
# formatted data saved in a seperate df to preserve the original numerical data 
summary_df = df[["School", "Type", "Total Students", "Total Budget", "Budget Per Student", "Average Math Score", 
                 "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
summary_df.iloc[:,3:5] = summary_df.iloc[:,3:5].applymap("${:,.2f}".format)
summary_df.iloc[:,5:10] = round(summary_df.iloc[:,5:10],3)
summary_df.iloc[:,7:10] = summary_df.iloc[:,7:10].applymap("{}%".format)

summary_df

,School,Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.68%,81.933%,54.642%
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133%,97.04%,91.335%
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988%,80.739%,53.204%
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.31%,79.299%,54.29%
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392%,97.139%,90.599%
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753%,80.863%,53.528%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506%,96.253%,89.227%
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684%,81.316%,53.514%
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058%,81.222%,53.539%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595%,95.946%,90.541%


<a id='top_performing_schools'></a>
## Top Performing Schools
The top 5 performing schools based on % Overall Passing

In [5]:
# sort the school summary df by '% Overall Passing' column, in descending order, leaving only top 5 schools
top_performers = df.sort_values(by="% Overall Passing", ascending=False)
top_performers = top_performers.iloc[:5]

top_performers

,School,Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


<a id='bottom_performing_schools'></a>
## Bottom Performing Schools
The Bottom 5 performing schools based on % Overall Passing

In [6]:
# sort the school summary df by '% Overall Passing' column, in ascending order, leaving only bottom 5 schools
bottom_performers = df.sort_values(by="% Overall Passing", ascending=True)
bottom_performers = bottom_performers.iloc[:5]

bottom_performers

,School,Type,Total Students,Total Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


<a id='average_math_grade'></a>
## Average Math Score by Grade
A breakdown of average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school

In [7]:
# Dataframe to store the data
math_grade_data = pd.DataFrame()

# Data is grouped by grade and school and then math score averaged for that grade
ninth = combined_df[combined_df['grade'] == '9th' ].groupby(['school_name'])['math_score'].mean().reset_index()
tenth = combined_df[combined_df['grade'] == '10th' ].groupby(['school_name'])['math_score'].mean().reset_index()
eleventh = combined_df[combined_df['grade'] == '11th' ].groupby(['school_name'])['math_score'].mean().reset_index()
twelth = combined_df[combined_df['grade'] == '12th' ].groupby(['school_name'])['math_score'].mean().reset_index()

# Data appended to the df, formatted and displayed
math_grade_data['School'] = ninth.iloc[:,0]
math_grade_data['9th Grade'] = ninth.iloc[:,1]
math_grade_data['10th Grade'] = tenth.iloc[:,1]
math_grade_data['11th Grade'] = eleventh.iloc[:,1]
math_grade_data['12th Grade'] = twelth.iloc[:,1]
math_grade_data.iloc[:,:] = round(math_grade_data.iloc[:,:],3)

math_grade_data


,School,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,77.084,76.997,77.516,76.492
1,Cabrera High School,83.095,83.155,82.766,83.277
2,Figueroa High School,76.403,76.540,76.884,77.151
3,Ford High School,77.361,77.672,76.918,76.180
4,Griffin High School,82.044,84.229,83.842,83.356
5,Hernandez High School,77.438,77.337,77.136,77.187
6,Holden High School,83.787,83.430,85.000,82.855
7,Huang High School,77.027,75.909,76.447,77.226
8,Johnson High School,77.188,76.691,77.492,76.863
9,Pena High School,83.625,83.372,84.328,84.122


<a id='average_reading_grade'></a>
## Average Reading Score by Grade
A breakdown of average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school

In [8]:
# Dataframe to store the data
reading_grade_data = pd.DataFrame()

# Data is grouped by grade and school and then reading score averaged for that grade
ninth = combined_df[combined_df['grade'] == '9th' ].groupby(['school_name'])['reading_score'].mean().reset_index()
tenth = combined_df[combined_df['grade'] == '10th' ].groupby(['school_name'])['reading_score'].mean().reset_index()
eleventh = combined_df[combined_df['grade'] == '11th' ].groupby(['school_name'])['reading_score'].mean().reset_index()
twelth = combined_df[combined_df['grade'] == '12th' ].groupby(['school_name'])['reading_score'].mean().reset_index()

# Data appended to the df, formatted and displayed
reading_grade_data['School'] = ninth.iloc[:,0]
reading_grade_data['9th Grade'] = ninth.iloc[:,1]
reading_grade_data['10th Grade'] = tenth.iloc[:,1]
reading_grade_data['11th Grade'] = eleventh.iloc[:,1]
reading_grade_data['12th Grade'] = twelth.iloc[:,1]
reading_grade_data.iloc[:,:] = round(reading_grade_data.iloc[:,:],3)

reading_grade_data


,School,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,81.303,80.907,80.946,80.912
1,Cabrera High School,83.676,84.253,83.788,84.288
2,Figueroa High School,81.199,81.409,80.640,81.385
3,Ford High School,80.633,81.263,80.404,80.662
4,Griffin High School,83.369,83.707,84.288,84.014
5,Hernandez High School,80.867,80.660,81.396,80.857
6,Holden High School,83.677,83.325,83.816,84.699
7,Huang High School,81.290,81.512,81.417,80.306
8,Johnson High School,81.261,80.773,80.616,81.228
9,Pena High School,83.807,83.612,84.336,84.591


<a id='school_spending'></a>
## Scores by School Spending
A breakdown of school performance based on average spending per student

In [9]:
# CREATE BINS
bins = [0, 584.99, 629.99, 644.99, 675]

# CREATE BIN NAMES
group_names = ["<$585", "$585-629", "$630-644", "$645-675"]

# DATAFRAME - select only the data relating to school spending and student performance 
# 'SettingWithCopyWarning' can be ignored as the data is not altered, just grouped and displayed (hence this has been disabled at setup)
performance_data = df[['Budget Per Student', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# GROUPBY BINS
# extra column added with bin group names, then table created grouped by these names 
performance_data["Budget Category"] = pd.cut(performance_data["Budget Per Student"], bins, labels=group_names, include_lowest=True)
del performance_data["Budget Per Student"]
grouped_performance_data = performance_data.groupby("Budget Category")

# Formmatting and display 
performance_by_spending = grouped_performance_data.max().reset_index()
performance_by_spending.iloc[:,1:] = round(performance_by_spending.iloc[:,1:],3)
performance_by_spending.iloc[:,3:] = performance_by_spending.iloc[:,3:].applymap("{}%".format)
performance_by_spending


,Budget Category,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,<$585,83.803,83.989,94.133%,97.04%,91.335%
1,$585-629,83.840,84.045,94.595%,97.139%,90.599%
2,$630-644,83.418,83.849,93.272%,97.309%,90.948%
3,$645-675,77.290,81.183,66.753%,81.316%,53.539%


<a id='school_size'></a>
## Scores by School Size
A breakdown of school performance based on schools size

In [10]:
# CREATE BINS
bins = [0, 999, 1999, 5000]

# CREATE BIN NAMES
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# DATAFRAME - select only the data relating to school spending and student performance 
# 'SettingWithCopyWarning' can be ignored as the data is not altered, just grouped and displayed (hence this has been disabled at setup)
performance_data = df[['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# GROUPBY BINS
# extra column added with bin group names, then table created grouped by these names 
performance_data["Student No. Category"] = pd.cut(performance_data["Total Students"], bins, labels=group_names, include_lowest=True)
del performance_data["Total Students"]
grouped_performance_data = performance_data.groupby("Student No. Category")

# FORMATTING AND DISPLAY
performance_by_size = grouped_performance_data.max().reset_index()
performance_by_size.iloc[:,1:] = round(performance_by_size.iloc[:,1:],3)
performance_by_size.iloc[:,3:] = performance_by_size.iloc[:,3:].applymap("{}%".format)
performance_by_size



,Student No. Category,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Small (<1000),83.840,84.045,94.595%,96.253%,90.541%
1,Medium (1000-2000),83.682,83.976,94.133%,97.309%,91.335%
2,Large (2000-5000),83.274,83.989,93.868%,96.54%,90.583%


<a id='school_type'></a>
## Scores by School Type
A breakdown of school performance based on school type

In [ ]:
# CREATE BINS
bins = [0, 999, 1999, 5000]

# CREATE BIN NAMES
group_names = ["District", "Charter"]





# # DATAFRAME - select only the data relating to school spending and student performance 
# # 'SettingWithCopyWarning' can be ignored as the data is not altered, just grouped and displayed (hence this has been disabled at setup)
# performance_data = df[['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# # GROUPBY BINS
# # extra column added with bin group names, then table created grouped by these names 
# performance_data["Student No. Category"] = pd.cut(performance_data["Total Students"], bins, labels=group_names, include_lowest=True)
# del performance_data["Total Students"]
# grouped_performance_data = performance_data.groupby("Student No. Category")

# # FORMATTING AND DISPLAY
# performance_by_size = grouped_performance_data.max().reset_index()
# performance_by_size.iloc[:,1:] = round(performance_by_size.iloc[:,1:],3)
# performance_by_size.iloc[:,3:] = performance_by_size.iloc[:,3:].applymap("{}%".format)
# performance_by_size


<a id='school_summary_alt'></a>
## School Summary - Alternative Solution
Alternative technique to producing a summary of school data using iteration

In [11]:
# list declaratations 
name, school_type, total_students, total_budget, per_student_budget, average_math_score, \
average_reading_score, percent_passing_math, percent_passing_reading, percent_passing_both = ([] for i in range(10))

# iterate through schools list
for school in school_data['school_name']:
    
    # create df for current school
    school_df = combined_df[combined_df['school_name'] == school]
    
    # current school name and append to list
    name.append(school)
    
    # get school type and append to list
    school_type.append(school_df.iloc[0,8])

    # get total students for school and append to list 
    total_students.append(school_df.iloc[0,9])
    
    # get budget for school and append to list. value displayed with currency formatting 
    budget_data = school_df.iloc[0,10]
    total_budget.append(budget_data)
    
    # get budget per student (total budget / total students) and append to list
    budget_data = (school_df.iloc[0,10])/(school_df.iloc[0,9])
    per_student_budget.append(budget_data)
    
    # get average math score for school and append to list
    average_math_score.append(round((school_df['math_score'].mean()),3))
    
    # get average reading score for school and append to list
    average_reading_score.append(round((school_df['reading_score'].mean()),3))
    
    # get percentage that passed math (>= 70%) for current school and append to list
    num_passing_math = school_df[school_df['math_score'] >= 70]
    percent_passing_math.append(f"{round((len(num_passing_math)/len(school_df)*100),3)}%")
          
    # get percentage that passed reading (>= 70%) for current school and append to list
    num_passing_reading = school_df[school_df['reading_score'] >= 70]
    percent_passing_reading.append(f"{round((len(num_passing_reading)/len(school_df)*100),3)}%")          
        
    # get percentage that passed both
    num_passing_both = num_passing_math[num_passing_math['reading_score'] >= 70]
    percent_passing_both.append(f"{round((len(num_passing_both)/len(school_df)*100),3)}%")

# merge results into a dataframe and display
school_summary = pd.DataFrame({"School": name, "School Type": school_type, "Total Students": total_students,
                               "Total School Budget": total_budget, "Per Student Budget": per_student_budget, 
                               "Average Math Score": average_math_score, "Average Reading Score": average_reading_score, 
                               "% Passing Math": percent_passing_math, "% Passing Reading": percent_passing_reading, 
                               "% Overall Passing": percent_passing_both})

school_summary



,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,76.629,81.183,65.684%,81.316%,53.514%
1,Figueroa High School,District,2949,1884411,639.0,76.712,81.158,65.988%,80.739%,53.204%
2,Shelton High School,Charter,1761,1056600,600.0,83.359,83.726,93.867%,95.855%,89.892%
3,Hernandez High School,District,4635,3022020,652.0,77.290,80.934,66.753%,80.863%,53.528%
4,Griffin High School,Charter,1468,917500,625.0,83.351,83.817,93.392%,97.139%,90.599%
5,Wilson High School,Charter,2283,1319574,578.0,83.274,83.989,93.868%,96.54%,90.583%
6,Cabrera High School,Charter,1858,1081356,582.0,83.062,83.976,94.133%,97.04%,91.335%
7,Bailey High School,District,4976,3124928,628.0,77.048,81.034,66.68%,81.933%,54.642%
8,Holden High School,Charter,427,248087,581.0,83.803,83.815,92.506%,96.253%,89.227%
9,Pena High School,Charter,962,585858,609.0,83.840,84.045,94.595%,95.946%,90.541%
